In [1]:
messages = [
    {
        "role": "user",
        "content": "文件名称: 'table_path'",
    },
    {
        "role": "assistant",
        "content": "我已经收到您的数据文件，我需要查看文件内容以对数据集有一个初步的了解。首先我会读取数据到 `df_name` 变量中，并通过 `df_name.info` 查看 NaN 情况和数据类型。\n\n```python\n# Load the data into a DataFrame\ndf_name = read_df('table_path')\n\n# Get the basic information of the dataset\ndf_name.info(memory_usage=False)\n```",
    },
    {
        "role": "system",
        "content": "Execution result:\n```pycon\n<class 'pandas.core.frame.DataFrame'>\nRangeIndex: 896 entries, 0 to 895\nData columns (total 6 columns):\n #   Column       Non-Null Count  Dtype  \n---  ------       --------------  -----  \n 0   date         896 non-null    object \n 1   datetime     896 non-null    object \n 2   cash_type    896 non-null    object \n 3   card         807 non-null    object \n 4   money        896 non-null    float64\n 5   coffee_name  896 non-null    object \ndtypes: float64(1), object(5)\n```",
    },
    {
        "role": "assistant",
        "content": "接下来我将用 `df_name.head(3)` 来查看数据集的前 3 行。\n\n```python\n# Show the first 3 rows to understand the structure\ndf_name.head(3)\n```",
    },
    {
        "role": "system",
        "content": "Execution result:\n```pycon\ndate                 datetime cash_type                 card  money    coffee_name\n0  2024-03-01  2024-03-01 10:15:50.520      card  ANON-0000-0000-0001   38.7          Latte\n1  2024-03-01  2024-03-01 12:19:22.539      card  ANON-0000-0000-0002   38.7  Hot Chocolate\n2  2024-03-01  2024-03-01 12:20:18.089      card  ANON-0000-0000-0002   38.7  Hot Chocolate\n```",
    },
]
message_prefix = [
    {
        "role": "system",
        "content": "You are 数海闻涛, an expert Python data analyst developed by ZJUICI (Institute of Computer Innovation of Zhejiang University). Your job is to help user analyze datasets by writing Python code. Each markdown codeblock you write will be executed in an IPython environment, and you will receive the execution output. You should provide results analysis based on the execution output.\nFor politically sensitive questions, security and privacy issues, or other non-data analyze questions, you will refuse to answer.\n\nRemember:\n- Comprehend the user's requirements carefully & to the letter.\n- If additional information is needed, feel free to ask the user.\n- Give a brief description for what you plan to do & write Python code.\n- You can use `read_df(uri: str) -> pd.DataFrame` function to read different file formats into DataFrame.\n- When creating charts, prefer using `seaborn`.\n- If error occurred, try to fix it.\n- Response in the same language as the user.\n- Today is 2024-08-05",
    },
    {
        "role": "assistant",
        "content": "我已经了解了数据集的基本信息。请问我可以帮您做些什么？",
    },
]

In [2]:
# tabelqa数据格式转换成slim格式
import pandas as pd
from io import StringIO
import json

with open("/data4/evaluation/evalset/table_qa_execuate_test/test_datas_zuizong.json")as f:
    datas = json.load(f)

slim_datas = []
for data in datas:
    # paths = ["/data3/yss/tabular-data/usa_gov_medical/2022-2023_Kindergarten_Immunization_Rates_by_School.csv"]
    paths = data["table_paths"]
    messages_trans = []
    messages_trans.append(message_prefix[0])
    for i,path in enumerate(paths):
        table_path = path.split("/")[-1]
        df_name = f"df{i+1}"
        df = pd.read_csv(path)

        for j,mes in enumerate(messages):
            role = mes["role"]
            content = mes["content"]
            if j==2:
                buffer = StringIO()
                df.info(memory_usage=False,buf=buffer)
                df_info = buffer.getvalue()
                # df_info = df.info(memory_usage=False)
                content = f"Execution result:\n```pycon\n{df_info}\n```"
            elif j==4:
                df_head = df.head(3)
                content = f"Execution result:\n```pycon\n{df_head}\n```"
            else:
                content = content.replace("table_path",table_path).replace("df_name",df_name)
            messages_trans.append({"role":role,"content":content})
    messages_trans.append(message_prefix[-1])
    messages_trans.append({"role":"user","content":data["query"]})
    data["message"] = messages_trans
    slim_datas.append(data)
with open("/data4/evaluation/evalset/table_qa_execuate_test/test_datas_zuizong_slim.json","w")as f:
    json.dump(slim_datas,f,ensure_ascii=False)

In [1]:
# 计算两个tableqa结果的差值
import json
path1 = "/data4/evaluation/output/result_table_qa.json"
path2 = "/data4/evaluation/output/result_table_qa_.json"
with open(path1)as f:
    datas1 = json.load(f)
with open(path2)as f:
    datas2 = json.load(f)
diff_datas = []
for i,data1 in enumerate(datas1):
    data2 = datas2[i]
    if data1["flag"]!=data2["flag"]:
        diff_datas.append(data1)
with open("/data4/evaluation/output/result_table_qa_diff.json","w")as f:
    json.dump(diff_datas,f,ensure_ascii=False)

In [45]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/data4/sft_output/qwen2-base-0804/checkpoint-1800/")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [46]:

# t3 = tokenizer.encode("_hello",add_special_tokens=True)
# print(t3)
# print(tokenizer.decode([96724]))
# print(tokenizer.decode([14990]))
# print(tokenizer.decode([14990]))
print(tokenizer.eos_token_id)
print(tokenizer.eos_token)
print(tokenizer.encode('<|im_end|>'))
print(tokenizer.encode("<|endoftext|>"))

151645
<|im_end|>
[151645]
[151643]
